#### Q1: Getting the embeddings model

Sentence Transformers documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [4]:
user_question = "I just discovered the course. Can I still join it?"
model.encode(user_question)[0]

0.07822265

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [6]:
len(documents)

948

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [8]:
filtered_documents = []
for doc in documents:
    if doc["course"] == "machine-learning-zoomcamp":
        filtered_documents.append(doc)

In [9]:
len(filtered_documents)

375

### Q2. Creating the embeddings

In [10]:
embeddings = []
for doc in filtered_documents:
    qa_text = f'{doc["question"]} {doc["text"]}'
    embeddings.append(model.encode(qa_text))

In [11]:
import numpy as np

In [12]:
X = np.array(embeddings)
X.shape

(375, 768)

### Q3. Search

In [13]:
query_vector = model.encode(user_question)
scores = X.dot(query_vector)

#### What's the highest score in the results?

In [14]:
max(scores)

0.6506572

### Vector search
We can now compute the similarity between a query vector and all the embeddings.

In [15]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(query_vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4. Hit-rate for our search engine

In [16]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [25]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [22]:
from tqdm.auto import tqdm

In [23]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    query_vector = model.encode(q['question'])
    results = search_engine.search(query_vector, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [27]:
relevance_total[:5]

[[False, False, False, False, False],
 [True, False, False, False, False],
 [True, False, False, False, False],
 [False, False, False, False, False],
 [False, False, False, True, False]]

In [28]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [29]:
hit_rate(relevance_total)

0.9398907103825137

### Q5. Indexing with Elasticsearch
Now let's index these documents with elasticsearch

Create the index with the same settings as in the module (but change the dimensions)
Index the embeddings (note: you've already computed them)
After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [30]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '2e800841c5b6', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'NYhrpSFxSpmULXfrDMAljQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [40]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [43]:
for doc in tqdm(filtered_documents):
    doc['question_text_vector'] = model.encode(doc['question'] + ' ' + doc['text'])

  0%|          | 0/375 [00:00<?, ?it/s]

In [44]:
for doc in tqdm(filtered_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [56]:
query = {
    "field": "question_text_vector",
    "query_vector": query_vector,
    "k": 5,
    "num_candidates": 10000, 
}

In [58]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course", "id"]) 
res["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': 'FVgrt5AB2WRTCCo7sIWE',
 '_score': 0.71432114,
 '_source': {'question': 'Any advice for adding the Machine Learning Zoomcamp experience to your LinkedIn profile?',
  'course': 'machine-learning-zoomcamp',
  'section': 'Miscellaneous',
  'text': 'I’ve seen LinkedIn users list DataTalksClub as Experience with titles as:\nMachine Learning Fellow\nMachine Learning Student\nMachine Learning Participant\nMachine Learning Trainee\nPlease note it is best advised that you do not list the experience as an official “job” or “internship” experience since DataTalksClub did not hire you, nor financially compensate you.\nOther ways you can incorporate the experience in the following sections:\nOrganizations\nProjects\nSkills\nFeatured\nOriginal posts\nCertifications\nCourses\nBy Annaliese Bronz\nInteresting question, I put the link of my project into my CV as showcase and make posts to show my progress.\nBy Ani Mkrtumyan',
  'id': 'c6a22665'}}

### Q6. Hit-rate for Elasticsearch
The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

In [81]:
relevance_total_elastic = []
for q in tqdm(ground_truth):
    doc_id = q['document']
    query_vector = model.encode(q['question'])
    query = {
        "field": "question_text_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 10000, 
    }
    es_results = es_client.search(index=index_name, knn=query,
                        source=["text", "section", "question", "course", "id"])
    relevance = [d['_source']['id'] == doc_id for d in es_results['hits']['hits']]
    relevance_total_elastic.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [82]:
hit_rate(relevance_total_elastic)

0.9398907103825137